In [ ]:
import os
import shutil
import yaml
import time
import json
import cv2
import random
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold, StratifiedKFold
from tqdm.notebook import tqdm
import seaborn as sns
import torch
from IPython.display import Image, clear_output
from collections import Counter
from ensemble_boxes import *
import copy
import os.path as osp
import mmcv
import mmdet
import numpy as np
import albumentations as A
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset
from mmcv import Config
from mmdet.apis import set_random_seed
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
print(mmdet.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
VER = 'v0'
TEST = False
DEBUG = False
PARAMS = {
    'version': VER,
    'folds': 4,
    'val_fold': 0,
    'img_size': 640,
    'batch_size': 4,
    'epochs': 10,
    'seed': 2020,
    'iou_th': .6,
    # 0
    #'config': 'faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_1x_coco.py',    
    # 1
    #'config': 'faster_rcnn/faster_rcnn_r50_caffe_fpn_mstrain_3x_coco.py',
    #'checkpoint': 'mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth',
    # 2
    'config': 'vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco.py',
    'checkpoint': 'vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth',
    # 3
    #'config': 'vfnet/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco.py',
    #'checkpoint': 'vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth',
    #'split_mkf': True,
    'comments': ''
}
DATA_PATH = '/u01/mrorange/siim/data'
WRK_DIR = f'/u01/mrorange/siim/mmdetection'
IMGS_PATH = f'{DATA_PATH}/train_{PARAMS["img_size"]}'
MDLS_PATH = f'/u01/mrorange/siim/models_mmdet_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
with open(f'{MDLS_PATH}/params.json', 'w') as file:
    json.dump(PARAMS, file)
    
def seed_all(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_all(PARAMS['seed'])
start_time = time.time()

In [ ]:
if TEST: 
    !mkdir {WRK_DIR}/checkpoints
    !wget -c http://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
          -O {WRK_DIR}/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth
    !wget -c https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth \
        -O {WRK_DIR}/checkpoints/vfnet_r50_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-6879c318.pth
    !wget -c https://openmmlab.oss-cn-hangzhou.aliyuncs.com/mmdetection/v2.0/vfnet/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth \
        -O {WRK_DIR}/checkpoints/vfnet_r101_fpn_mdconv_c3-c5_mstrain_2x_coco_20201027pth-7729adb5.pth
    
    config = f'{WRK_DIR}/configs/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco.py'
    checkpoint = f'{WRK_DIR}/checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth'
    model = init_detector(config, checkpoint, device='cuda:0')
    img = 'tomatokillers.jpg'
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result, score_thr=.5)
else:
    print('no test mode')

In [ ]:
train_df = pd.read_csv(f'{DATA_PATH}/train_meta_{PARAMS["img_size"]}.csv')
if DEBUG:
    train_df = train_df.loc[:100]
df_train_img = pd.read_csv(f'{DATA_PATH}/train_image_level.csv')
df_train_sty = pd.read_csv(f'{DATA_PATH}/train_study_level.csv')

train_df['id'] = train_df['img'].apply(lambda x: x.split('/')[-1].replace('.png', '_image'))
df_train_sty['StudyInstanceUID'] = df_train_sty['id'].apply(lambda x: x.replace('_study', ''))
del df_train_sty['id']
df_train_img = df_train_img.merge(df_train_sty, on='StudyInstanceUID')
train_df = df_train_img.merge(train_df, on='id')
print(train_df.shape)
display(train_df.head())

In [ ]:
def bar_plot(train_df, variable):
    var = train_df[variable]
    varValue = var.value_counts()
    plt.figure(figsize = (12, 3))
    plt.bar(varValue.index, varValue)
    plt.xticks(varValue.index, varValue.index.values)
    plt.ylabel("Frequency")
    plt.title(variable)
    plt.show()
    print("{}: \n{}".format(variable, varValue))

train_df['target'] = 'Negative for Pneumonia'
train_df.loc[train_df['Typical Appearance']==1, 'target'] = 'Typical Appearance'
train_df.loc[train_df['Indeterminate Appearance']==1, 'target'] = 'Indeterminate Appearance'
train_df.loc[train_df['Atypical Appearance']==1, 'target'] = 'Atypical Appearance'
bar_plot(train_df, 'target') 

In [ ]:
train_df = train_df[~train_df.boxes.isnull()] 
train_df.reset_index(inplace=True)
classes = [
    'Typical Appearance', 
    'Indeterminate Appearance', 
    'Atypical Appearance'
]
print('classes:\n', classes,
      '\nclasses labels:\n', np.unique(train_df[classes].values, axis=0))

In [ ]:
label2color = {
    '[1, 0, 0]': [255, 0, 0], # Typical Appearance
    '[0, 1, 0]': [0, 255, 0], # Indeterminate Appearance
    '[0, 0, 1]': [0, 0, 255], # Atypical Appearance
}
label2classes = {
    '[1, 0, 0]': classes[0],
    '[0, 1, 0]': classes[1],
    '[0, 0, 1]': classes[2]
}

def plot_img(img, size=(18, 18), title='', cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title='', cmap='gray', img_size=None):
    rows = len(imgs) // cols + 1
    fig = plt.figure(figsize=(cols * size, rows * size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i + 1)
        plt.axis('off')
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.axis('off')
    
def draw_bbox(img, box, label, color, thickness=3):   
    alpha = .1
    alpha_box = .4
    overlay_bbox = img.copy()
    overlay_text = img.copy()
    output = img.copy()
    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, .6, 1)[0]
    cv2.rectangle(overlay_bbox, 
                  (box[0], box[1]), 
                  (box[2], box[3]), 
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    cv2.rectangle(overlay_text, 
                  (box[0], box[1] - 7 - text_height), 
                  (box[0] + text_width + 2, box[1]),
                  (0, 0, 0), -1)
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.rectangle(output, 
                  (box[0], box[1]), 
                  (box[2], box[3]),
                  color, thickness)
    cv2.putText(output, 
                label.upper(), 
                (box[0], box[1]-5),
                cv2.FONT_HERSHEY_SIMPLEX, 
                .6, (255, 255, 255), 1, 
                cv2.LINE_AA)
    return output

In [ ]:
imgs = []
sample = train_df.sample(n=6)['img'].values
for img_name in sample:
    img = cv2.imread(f'{IMGS_PATH}/{img_name}')
    imgs.append(img)
plot_imgs(imgs, size=8, cols=len(sample), cmap=None)

In [ ]:
imgs = []
sample = train_df.sample(n=4)['img'].values
for img_name in sample:
    ratio_x = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim_x'].values[0]
    ratio_y = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim_y'].values[0]
    boxes = train_df.loc[train_df['img'] == img_name, 'boxes'].values[0]
    boxes = json.loads(boxes.replace('\'', '\"'))
    boxes = [[int(box['x'] * ratio_x), 
              int(box['y'] * ratio_y), 
              int((box['x'] + box['width']) * ratio_x), 
              int((box['y'] + box['height']) * ratio_y)]
             for box in boxes]
    img_labels = train_df.loc[train_df['img'] == img_name, classes].values[0]
    img_labels = [str(img_labels.tolist())] * len(boxes)
    img = cv2.imread(f'{IMGS_PATH}/{img_name}')
    for label_id, box in zip(img_labels, boxes):
        color = label2color[label_id]
        img = draw_bbox(
            img, 
            list(np.int_(box)), 
            label2classes[label_id], 
            label2color[label_id]
        )
    imgs.append(img)
plot_imgs(imgs, size=8, cols=4, cmap=None)

In [ ]:
skf  = StratifiedKFold(n_splits=PARAMS['folds'])
train_df['fold'] = -1
for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, y=train_df.target)):
    train_df.loc[val_idx, 'fold'] = fold

In [ ]:
train_df.loc[train_df.fold == 0, 'target'].value_counts()

In [ ]:
train_df.loc[train_df.fold != 0, 'target'].value_counts()

In [ ]:
split = PARAMS['val_fold']
with open(f'{MDLS_PATH}/train.txt', 'w') as file:
    tr_ids = list(train_df[train_df['fold'] != split].img.unique())
    print('train:', len(tr_ids))
    file.write('\n'.join(tr_ids))
with open(f'{MDLS_PATH}/val.txt', 'w') as file:
    val_ids = list(train_df[train_df['fold'] == split].img.unique())
    print('val:', len(val_ids))
    file.write('\n'.join(val_ids))

In [ ]:
@DATASETS.register_module()
class SIIMDataset(CustomDataset):
    #CLASSES = classes.copy()
    CLASSES = ('opacity', )
    ANN_DF = train_df.copy()
    def load_annotations(self, ann_file):
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
        for img_id in image_list:
            img_anns = self.ANN_DF[self.ANN_DF.img == img_id]
            filename = f'{self.img_prefix}/{img_anns["img"].values[0]}'
            data_info = dict(
                filename=filename, 
                width=PARAMS['img_size'], 
                height=PARAMS['img_size']
            )
            ratio_x = PARAMS['img_size'] / img_anns['dim_x'].values[0]
            ratio_y = PARAMS['img_size'] / img_anns['dim_y'].values[0]
            boxes = img_anns['boxes'].values[0]
            boxes = json.loads(boxes.replace('\'', '\"'))
            gt_bboxes = [
                [int(box['x'] * ratio_x), 
                 int(box['y'] * ratio_y), 
                 int((box['x'] + box['width']) * ratio_x), 
                 int((box['y'] + box['height']) * ratio_y)]
                for box in boxes]
            img_labels = img_anns[classes].values[0]
            #gt_labels = [cat2label[label2classes[str(img_labels.tolist())]]] * len(boxes)
            gt_labels = [0] * len(boxes)
            data_anno = dict(
                bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                labels=np.array(gt_labels),
            )
            data_info.update(ann=data_anno)
            #print(data_anno)
            data_infos.append(data_info)
        return data_infos

In [ ]:
train_transforms = A.Compose([
    A.OneOf([
        A.RandomBrightness(limit=.2, p=1), 
        A.RandomContrast(limit=.2, p=1), 
        A.RandomGamma(p=1)
    ], p=.5),
    A.OneOf([
        A.Blur(blur_limit=3, p=1),
        A.MedianBlur(blur_limit=3, p=1)
    ], p=.25),
    A.OneOf([
        A.GaussNoise(0.002, p=.5),
        A.IAAAffine(p=.5),
    ], p=.25),
    A.VerticalFlip(p=.5),
    A.HorizontalFlip(p=.5),
    A.Transpose(p=.25),
    A.RandomRotate90(p=.25),
    A.Cutout(num_holes=10, max_h_size=20, max_w_size=20, p=.25),
    A.ShiftScaleRotate(p=.5)
])

In [ ]:
cfg = Config.fromfile(f'{WRK_DIR}/configs/{PARAMS["config"]}')
cfg.load_from = f'{WRK_DIR}/checkpoints/{PARAMS["checkpoint"]}'
#cfg.model.roi_head.bbox_head.num_classes = len(classes)
#cfg.model.bbox_head.num_classes = len(classes) # VFNet option
cfg.model.bbox_head.num_classes = 1
#cfg.model.rpn_head.loss_bbox=dict(
#    type='IoULoss', 
#    loss_weight=1.0)
cfg.dump(f'{MDLS_PATH}/init_config.py')

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(1333, 640), (1333, 672), (1333, 704), (1333, 736),
                   (1333, 768), (1333, 800)],
        multiscale_mode='value',
        keep_ratio=True),
    #dict(type='RandomFlip', flip_ratio=0.25),
    ########################################
    # Note that this key is part of bbox_params. 
    # Their difference is format='pascal_voc' means [x1, y1, x2, y2] style box encoding, 
    # while format='coco' means [x, y, w, h].
    dict(
        type='Albu',
        transforms=train_transforms,
        bbox_params=dict(
            type='BboxParams',
            format='pascal_voc',
            label_fields=['gt_labels'],
            min_visibility=0.0,
            filter_lost_elements=True),
        keymap={
            'img': 'image',
            #'gt_masks': 'masks',
            'gt_bboxes': 'bboxes'},
        update_pad_shape=False,
        skip_img_without_anno=True),
    #########################################
    dict(
        type='Normalize',
        mean=[103.53, 116.28, 123.675],
        std=[1.0, 1.0, 1.0],
        to_rgb=False),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
]

cfg.dataset_type = 'SIIMDataset'
cfg.data_root = IMGS_PATH
cfg.data.test.type = 'SIIMDataset'
cfg.data.test.data_root = IMGS_PATH
cfg.data.test.ann_file = f'{MDLS_PATH}/train.txt'
cfg.data.test.img_prefix = ''
cfg.data.train.type = 'SIIMDataset'
cfg.data.train.data_root = IMGS_PATH
cfg.data.train.ann_file = f'{MDLS_PATH}/train.txt'
cfg.data.train.img_prefix = ''
cfg.data.val.type = 'SIIMDataset'
cfg.data.val.data_root = IMGS_PATH
cfg.data.val.ann_file = f'{MDLS_PATH}/val.txt'
cfg.data.val.img_prefix = ''
cfg.work_dir = MDLS_PATH

cfg.optimizer.lr = .02 / (8 * 16 / PARAMS['batch_size'])
#cfg.optimizer = dict(type='Adam', lr=.001)
#cfg.optimizer_config = dict(grad_clip=dict(max_norm=35, norm_type=2))
#cfg.lr_config = dict(
#    policy='CosineAnnealing',
#    warmup='exp',
#    warmup_iters=500,
#    warmup_ratio=.1,
#    min_lr_ratio=1e-5
#)

cfg.log_config.interval = 128
cfg.runner.max_epochs = PARAMS['epochs']
cfg.checkpoint_config.interval = 1
cfg.evaluation = dict(
    interval=1, 
    start=2,
    metric='mAP', 
    save_best='mAP')

cfg.seed = PARAMS['seed']
set_random_seed(0, deterministic=False)

cfg.gpu_ids = range(1)
cfg.data.samples_per_gpu = PARAMS['batch_size']
cfg.data.workers_per_gpu = 2 * PARAMS['batch_size']
#cfg.workflow = [('train', 1), ('val', 1)]
cfg.workflow = [('train', 1)]

cfg.dump(f'{MDLS_PATH}/train_config.py')
print(f'Config:\n{cfg.pretty_text}')

In [ ]:
datasets = [build_dataset(cfg.data.train)]
if len(cfg.workflow) == 2:
    datasets.append(build_dataset(cfg.data.val))
model = build_detector(
    cfg.model, 
    train_cfg=cfg.get('train_cfg'), 
    test_cfg=cfg.get('test_cfg')
)
model.CLASSES = datasets[0].CLASSES
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

In [ ]:
checkpoint = f'{MDLS_PATH}/epoch_6.pth'
cfg = f'{MDLS_PATH}/init_config.py'
model_test = init_detector(cfg, checkpoint, device='cuda:0')

In [ ]:
TH = .45
imgs = []
split = PARAMS['val_fold']
sample = train_df[train_df['fold'] != split].sample(n=4)['img'].values
for img_name in sample:
    ratio_x = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim_x'].values[0]
    ratio_y = PARAMS['img_size'] / train_df.loc[train_df['img'] == img_name, 'dim_y'].values[0]
    boxes = train_df.loc[train_df['img'] == img_name, 'boxes'].values[0]
    boxes = json.loads(boxes.replace('\'', '\"'))
    boxes = [[int(box['x'] * ratio_x), 
              int(box['y'] * ratio_y), 
              int((box['x'] + box['width']) * ratio_x), 
              int((box['y'] + box['height']) * ratio_y)]
             for box in boxes]
    img_labels = train_df.loc[train_df['img'] == img_name, classes].values[0]
    img_labels = [str(img_labels.tolist())] * len(boxes)
    img = cv2.imread(f'{IMGS_PATH}/{img_name}')
    for label_id, box in zip(img_labels, boxes):
        color = label2color[label_id]
        img = draw_bbox(
            img, 
            list(np.int_(box)), 
            label2classes[label_id], 
            label2color[label_id]
        )
    result = inference_detector(model_test, img)
    boxes_list = [list(x[:, :4] / PARAMS['img_size']) for x in result if x.shape[0] != 0]
    boxes_list =  [item for sublist in boxes_list for item in sublist]
    scores_list = [x[:, 4].tolist() for x in result if x.shape[0] != 0]
    scores_list =  [item for sublist in scores_list for item in sublist]
    labels_list = [[i] * x.shape[0] for i, x in enumerate(result) if x.shape[0] != 0]
    labels_list =  [item for sublist in labels_list for item in sublist]
    boxes, scores, box_labels = nms(
        boxes=[boxes_list], 
        scores=[scores_list], 
        labels=[labels_list], 
        weights=None,
        iou_thr=PARAMS['iou_th']
    )
    boxes *= PARAMS['img_size']
    for label_id, box, score in zip(box_labels, boxes, scores):
        if score >= TH:
            color = [255, 255, 255]
            img = draw_bbox(
                img, 
                list(np.int_(box)), 
                'predict', 
                color
            )
    imgs.append(img)
plot_imgs(imgs, size=8, cols=4, cmap=None)